# **Khởi tạo trang web > Đăng nhập vào trang web > Lấy link bài đăng**

In [ ]:
#!pip install selenium
import csv
import pandas as pd
from selenium import webdriver
import os
import time
import random
import math
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import requests
import numpy as np
from bs4 import BeautifulSoup
def driver_chrome():
    os.environ['PATH'] += r"E:\Video a trung\chromedriver-win64 (1)\chromedriver-win64\chromedriver.exe"
    WINDOW_SIZE = "1920,1080"
    chrome_options = Options()
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument("--disable-extensions")  # Tắt phần mở rộng
    chrome_options.add_argument("--disable-popup-blocking")  # Tắt chặn popup
    chrome_options.add_argument("--incognito")  # Chế độ ẩn danh
    chrome_options.add_argument("--remote-debugging-port=9222")  # Debug Chrome
    chrome_options.add_argument("--disable-blink-features=AutomationControllered")
    chrome_options.add_experimental_option('useAutomationExtension', False)
    prefs = {
    "profile.default_content_setting_values.notifications": 2,  # Tắt thông báo
    "credentials_enable_service": False,  # Tắt dịch vụ lưu thông tin đăng nhập
    "profile.password_manager_enabled": False}  # Tắt trình quản lý mật khẩu
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--start-maximized")  # open Browser in maximized mode
    chrome_options.add_argument("--disable-dev-shm-usage")  # overcome limited resource problems
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_argument('disable-infobars')
    driver = webdriver.Chrome(options=chrome_options)
    return driver
def random_delay(min_seconds, max_seconds):
    time.sleep(random.uniform(min_seconds, max_seconds))
def readData(filePath):
    try:
        with open(filePath, 'r') as file:
            reader = csv.reader(file)
            data = [row[0] for row in reader]
        return data
    except FileNotFoundError:
        return []

def writeFileTxt(filePath, postId):
    with open(filePath, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([postId])
def scroll_and_wait(driver, last_height):
    while True:
        # Cuộn xuống cuối trang
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # Chờ trang tải hoàn chỉnh
        time.sleep(2)

        # Lấy chiều cao mới sau khi cuộn
        new_height = driver.execute_script("return document.body.scrollHeight")

        # Kiểm tra nếu đã cuộn đến cuối trang
        if new_height == last_height:
            break

        last_height = new_height
    time.sleep(1)
def get_job_links(driver, filePath):
    try:
        # Lấy tất cả các phần tử liên kết sau khi trang đã tải thêm
        all_link = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//div[@class="block-job-list"]/child::div'))
        )
        return all_link
    except Exception as e:
        print(f"Lỗi khi lấy các phần tử: {e}")
        return []
def getlink(driver,page, filePath = 'links.csv'):
    allLinks = readData(filePath)

    last_height = driver.execute_script("return document.body.scrollHeight")

    for i in range(1, page+ 1):  # Giả sử bạn có một biến `page_count` để xác định số trang
        driver.get(f"https://www.vietnamworks.com/viec-lam?page={i}")
        random_delay(2, 4)  # Giả sử bạn có một hàm random_delay để delay ngẫu nhiên
        time.sleep(2)

        # Cuộn trang và chờ trang tải thêm
        scroll_and_wait(driver, last_height)

        # Lấy danh sách các liên kết từ trang sau khi cuộn
        all_link = get_job_links(driver, filePath)

        if len(all_link) > 0:
            for j in range(len(all_link)-1):
                try:
                    # In ra số thứ tự của phần tử đang được xử lý
                    print(f"Đang xử lý phần tử thứ {j+1} trong tổng số {len(all_link)} phần tử.")

                    # Lấy phần tử <a> trong từng phần tử
                    link_id = all_link[j].find_element(By.TAG_NAME, 'a')

                    # Kiểm tra xem phần tử <a> có tồn tại không
                    if link_id is not None:
                        link_cv = link_id.get_attribute('href')

                        # Kiểm tra nếu link hợp lệ và chưa có trong danh sách
                        if link_cv:
                            print(f"Link tìm được: {link_cv}")
                            if link_cv not in allLinks:
                                print(f"Link mới, ghi vào file: {link_cv}")
                                writeFileTxt(filePath, link_cv)  # Ghi vào file
                            else:
                                print(f"Link {link_cv} đã tồn tại trong danh sách.")
                        else:
                            print(f"Không tìm thấy href trong phần tử thứ {j+1}.")
                    else:
                        print(f"Không tìm thấy phần tử <a> trong phần tử thứ {j+1}.")

                except Exception as e:
                    print(f"Lỗi khi lấy link từ phần tử thứ {j+1}: {e}")


def connect_web(driver):
    # 1. Nhấn vào nút profile-button
    driver.get("https://www.vietnamworks.com/tim-viec-lam/tim-tat-ca-viec-lam")
    random_delay(2, 4)
    profile_button = driver.find_element(By.XPATH, '//button[@aria-label="profile-button"]')
    profile_button.click()
    random_delay(3, 5)  # Đợi 3-5 giây ngẫu nhiên

    # 2. Nhấn vào phần tử Google
    google_button = driver.find_element(By.XPATH, '//span[contains(text(), "Google")]')
    google_button.click()
    random_delay(3, 5)  # Đợi 3-5 giây ngẫu nhiên

    # 3. Nhập email
    email_field = driver.find_element(By.XPATH, '//*[@id="identifierId"]')
    email_field.send_keys("XXX") # Thay bằng gmail của bạn
    random_delay(2, 4)  # Đợi 1-3 giây ngẫu nhiên

    # 4. Nhấn nút "Tiếp theo" (Next)
    identifier_next_button = driver.find_element(By.XPATH, '//*[@id="identifierNext"]/div/button')
    identifier_next_button.click()
    random_delay(2, 7)  # Đợi 1-3 giây ngẫu nhiên

    # 5. Nhập mật khẩu
    password_field = driver.find_element(By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')
    password_field.send_keys("XXX") #Thay bằng mật khẩu của bạn
    random_delay(2, 4)  # Đợi 1-3 giây ngẫu nhiên

    # 6. Nhấn nút "Tiếp theo" (Next)
    password_next_button = driver.find_element(By.XPATH, '//*[@id="passwordNext"]/div/button/span')
    password_next_button.click()
    random_delay(3, 5)  # Đợi 1-3 giây ngẫu nhiên
    element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '(//span[@class="no-of-jobs"])[2]')))
    text = element.text  # Lấy text từ phần tử
    # Chuyển text sang số
    try:
        number = int(text.replace(",", "").strip())  # Loại bỏ dấu phẩy hoặc khoảng trắng trước khi chuyển đổi
    except ValueError:
        None
    page=number/50
    if page%1!=0:
        page=int(page)+1
    else:
        page=int(page)
    return page

driver=driver_chrome() # Khởi tạo chrome gồm các hành vi ẩn danh
page=connect_web(driver) # Kết nối với trang web và login vào
getlink(driver,page, filePath = 'links.csv') # Bắt đầu quét đường link và lưu tất cả đường link vào file links.csv


# **Tạo biến lưu trữ dữ liệu > Lấy dữ liệu > Xuất ra file excel**

In [ ]:
#!pip install selenium
import csv
import pandas as pd
import numpy as np
from selenium import webdriver
import os
import time
import random
import math
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import requests
from datetime import datetime, timedelta
import re
from bs4 import BeautifulSoup
def giatrirong():
    diadiem_cv,links,ten_cv, ngaydang_cv,hethan_cv, congty_cv, capbac_cv, mucluong_cv, loaihinh_cv, nganh_cv, mota_cv, yeucau_cv, phucloi_cv, tukhoa_cv, hocvan_cv, dotuoi_cv, soluong_cv, gio_cv, quoctich_cv, hoso_cv, gioitinh_cv, honnhan_cv, ngay_cv, kynang_cv, linhvuc_cv, luotxem_cv=[[] for _ in range(26)]
    return diadiem_cv,links,ten_cv, ngaydang_cv,hethan_cv, congty_cv, capbac_cv, mucluong_cv, loaihinh_cv, nganh_cv, mota_cv, yeucau_cv, phucloi_cv, tukhoa_cv, hocvan_cv, dotuoi_cv, soluong_cv, gio_cv, quoctich_cv, hoso_cv, gioitinh_cv, honnhan_cv, ngay_cv, kynang_cv, linhvuc_cv, luotxem_cv
def get_detail(driver,linkid,diadiem_cv,links,ten_cv, ngaydang_cv,hethan_cv, congty_cv, capbac_cv, mucluong_cv, loaihinh_cv, nganh_cv, mota_cv, yeucau_cv, phucloi_cv, tukhoa_cv, hocvan_cv, dotuoi_cv, soluong_cv, gio_cv, quoctich_cv, hoso_cv, gioitinh_cv, honnhan_cv, ngay_cv, kynang_cv, linhvuc_cv, luotxem_cv): # truyền vào số lượng comment cần lấy trên mỗi post nhé
    links.append(linkid)
    driver.get(f"{linkid}")
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_and_wait(driver, last_height)
    try:
        ten_cv.append(driver.find_element(By.XPATH, '//div/h1[@name="title"]').text)
    except Exception as e:
        ten_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy tên công việc: {e}")

    random_delay(1, 3)

    try:
        ngaydang_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "NGÀY ĐĂNG")]]').text)
    except Exception as e:
        ngaydang_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy ngày đăng bài: {e}")

    random_delay(2, 3)

    try:
        hethan = driver.find_element(By.XPATH, '(//span[contains(., "Hết hạn")])[1]')
        text = hethan.text  # "Hết hạn trong 11 ngày"
        # Tách số ngày từ chuỗi văn bản
        days_match = re.search(r'(\d+)', text)  # Tìm số trong chuỗi
        if days_match:
            days = int(days_match.group(1))  # Lấy số ngày
            # Cộng số ngày này vào ngày hiện tại
            expiration_date = datetime.now() + timedelta(days=days)
            # Chuyển đổi sang định dạng ngày tháng
            expiration_date_str = expiration_date.strftime("%d/%m/%Y")
            hethan_cv.append(expiration_date_str)
        else:
            hethan_cv.append('')  # Nếu không có ngày hết hạn, thêm chuỗi rỗng
    except Exception as e:
        hethan_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy ngày hết hạn: {e}")
    random_delay(1, 3)
    try:
        congty_cv.append(driver.find_element(By.XPATH, '//*[contains(., "Xem bản đồ")]/parent::div/parent::div/parent::div/preceding-sibling::a').text)
    except:
        try:
            congty_cv.append(driver.find_element(By.XPATH,'//a[@name="label"]').text)
        except Exception as e:
            congty_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
            print(f"Lỗi ở phần lấy tên công ty: {e}")

    random_delay(1, 4)
    try:
        diadiem_cv.append(driver.find_element(By.XPATH,'//*[contains(., "Xem bản đồ")]/preceding-sibling::p').text)
    except:
        try:
            diadiem_cv.append(driver.find_element(By.XPATH, '//*[contains(text(), "Địa điểm làm việc")]/following-sibling::div/descendant::p').text)
        except Exception as e:
            diadiem_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
            print(f"Lỗi ở phần lấy địa điểm: {e}")

    random_delay(2, 4)

    try:
        capbac_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "CẤP BẬC")]]').text)
    except Exception as e:
        capbac_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy cấp bậc: {e}")

    random_delay(2, 4)

    try:
        mucluong_cv.append(driver.find_element(By.XPATH, '(//div/h1[@name="title"]/ancestor::div/ancestor::div/following-sibling::div/child::div)[1]').text)
    except Exception as e:
        mucluong_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy mức lương: {e}")

    random_delay(1, 4)

    try:
        driver.find_element(By.XPATH,'(//button[@aria-label="Xem thêm"])[1]').click()  # Xem thêm phúc lợi
    except Exception as e:
        print(f"Lỗi ở phần Xem thêm phúc lợi: {e}")
    random_delay(2, 4)

    try:
        driver.find_element(By.XPATH,'(//button[@aria-label="Xem thêm"])[2]').click()  # Xem thêm thông tin công việc
    except Exception as e:
        print(f"Lỗi ở phần Xem thêm thông tin công việc: {e}")

    random_delay(2, 4)

    try:
        loaihinh_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "LOẠI HÌNH LÀM VIỆC")]]').text)
    except Exception as e:
        loaihinh_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy loại hình làm việc: {e}")

    random_delay(3, 4)

    try:
        nganh_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "NGÀNH NGHỀ")]]').text)
    except Exception as e:
        nganh_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy ngành nghề: {e}")

    random_delay(1, 4)
    try:
        driver.find_element(By.XPATH, '//button[@aria-label="Xem đầy đủ mô tả công việc"]').click()
    except Exception as e:
        print(f"Lỗi ở phần xem thêm mô tả công việc: {e}")
    random_delay(2, 3)
    try:
        mota_cv.append(driver.find_element(By.XPATH, '(//h2[contains(., "Mô tả công việc")])/following-sibling::div').text)
    except Exception as e:
        mota_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy mô tả công việc: {e}")
    random_delay(2, 3)
    try:
        yeucau_cv.append(driver.find_element(By.XPATH, '(//h2[contains(., "Yêu cầu công việc")])/following-sibling::div').text)
    except Exception as e:
        yeucau_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy yêu cầu công việc: {e}")

    random_delay(1, 2)
    try:
        phucloi = driver.find_elements(By.XPATH, '(//h2[contains(., "Các phúc lợi dành cho bạn")])/following-sibling::div/child::div/child::div/child::div')
        phuc_loi=[]
        for i in range(len(phucloi)):
            if i%2==0:
                phuc_loi.append(phucloi[i+1].text)
                #Phucloi = [Ploi.text for Ploi in phucloi[i]]
                # Tạo danh sách từ các giá trị text
        phucloi_cv.append('\n'.join(phuc_loi))
    except Exception as e:
        phucloi_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy phúc lợi: {e}")

    random_delay(3, 4)

    try:
        tukhoa = driver.find_elements(By.XPATH, '//div[contains(text(), "Từ khoá")]/../following-sibling::div//div//button')
        Tukhoa = [Tkhoa.text for Tkhoa in tukhoa]  # Tạo danh sách từ các giá trị text
        tukhoa_cv.append('\n'.join(Tukhoa))
    except Exception as e:
        tukhoa_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy từ khóa: {e}")

    random_delay(1, 3)

    try:
        hocvan_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "TRÌNH ĐỘ HỌC VẤN TỐI THIỂU")]]').text)
    except Exception as e:
        hocvan_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy trình độ học vấn tối thiểu: {e}")

    random_delay(2, 4)
    try:
        dotuoi_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "ĐỘ TUỔI MONG MUỐN")]]').text)
    except Exception as e:
        dotuoi_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy độ tuổi mong muốn: {e}")

    random_delay(2, 4)

    try:
        soluong_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "SỐ LƯỢNG TUYỂN DỤNG")]]').text)
    except Exception as e:
        soluong_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy số lượng tuyển dụng: {e}")

    random_delay(2, 3)

    try:
        gio_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "GIỜ LÀM VIỆC")]]').text)
    except Exception as e:
        gio_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy giờ làm việc: {e}")

    random_delay(1, 4)

    try:
        quoctich_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "QUỐC TỊCH")]]').text)
    except Exception as e:
        quoctich_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy quốc tịch: {e}")

    random_delay(1, 3)

    try:
        hoso_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "NGÔN NGỮ TRÌNH BÀY HỒ SƠ")]]').text)
    except Exception as e:
        hoso_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy ngôn ngữ trình bày hồ sơ: {e}")

    random_delay(2, 3)

    try:
        gioitinh_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "GIỚI TÍNH")]]').text)
    except Exception as e:
        gioitinh_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy giới tính: {e}")

    random_delay(1, 2)

    try:
        honnhan_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "TÌNH TRẠNG HÔN NHÂN")]]').text)
    except Exception as e:
        honnhan_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy tình trạng hôn nhân: {e}")

    random_delay(1, 3)

    try:
        ngay_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "NGÀY LÀM VIỆC")]]').text)
    except Exception as e:
        ngay_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy ngày làm việc: {e}")

    random_delay(2, 4)

    try:
        kynang_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "KỸ NĂNG")]]').text)
    except Exception as e:
        kynang_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy kỹ năng: {e}")

    random_delay(1, 4)

    try:
        linhvuc_cv.append(driver.find_element(By.XPATH, '//p[preceding-sibling::*[contains(., "LĨNH VỰC")]]').text)
    except Exception as e:
        linhvuc_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy lĩnh vực: {e}")

    random_delay(3, 4)

    try:
        luotxem_cv.append(driver.find_element(By.XPATH, '//span[contains(., "lượt xem")]').text)
    except Exception as e:
        luotxem_cv.append('')  # Thêm chuỗi rỗng nếu có lỗi
        print(f"Lỗi ở phần lấy lượt xem: {e}")
# Bắt đầu gọi các hàm
diadiem_cv,links,ten_cv, ngaydang_cv,hethan_cv, congty_cv, capbac_cv, mucluong_cv, loaihinh_cv, nganh_cv, mota_cv, yeucau_cv, phucloi_cv, tukhoa_cv, hocvan_cv, dotuoi_cv, soluong_cv, gio_cv, quoctich_cv, hoso_cv, gioitinh_cv, honnhan_cv, ngay_cv, kynang_cv, linhvuc_cv, luotxem_cv=giatrirong()
driver=driver_chrome()
for linkid in readData('links.csv'):
  get_detail(driver, linkid,diadiem_cv,links,ten_cv, ngaydang_cv,hethan_cv, congty_cv, capbac_cv, mucluong_cv, loaihinh_cv, nganh_cv, mota_cv, yeucau_cv, phucloi_cv, tukhoa_cv, hocvan_cv, dotuoi_cv, soluong_cv, gio_cv, quoctich_cv, hoso_cv, gioitinh_cv, honnhan_cv, ngay_cv, kynang_cv, linhvuc_cv, luotxem_cv)
df1 = pd.DataFrame(list(zip(ten_cv, ngaydang_cv,hethan_cv, congty_cv,diadiem_cv,links, capbac_cv, mucluong_cv, loaihinh_cv, nganh_cv,
                        mota_cv, yeucau_cv, phucloi_cv, tukhoa_cv, hocvan_cv, dotuoi_cv, soluong_cv, gio_cv, quoctich_cv, hoso_cv,
                        gioitinh_cv, honnhan_cv, ngay_cv, kynang_cv, linhvuc_cv, luotxem_cv)),
               columns = ['Tên công việc', 'Ngày đăng', 'Ngày Hết hạn', 'Công ty','Địa điểm','Đường dẫn','Chức vụ', 'Lương', 'Loại hình', 'Ngành nghề',
                          'Mô tả công việc', 'Yêu cầu công việc', 'Phúc lợi', 'Từ khóa', 'Học vấn', 'Độ tuổi', 'Số lượng',
                            'Giờ làm việc', 'Quốc tịch', 'Hồ sơ', 'Giới tính', 'Hôn nhân', 'Ngày làm việc', 'Kỹ năng', 'Lĩnh vực', 'Lượt xem'])
df1['index_']= np.arange(1, len(df1) + 1)
df1.to_excel('vietnamwork.xlsx', index=False)